### Test tìm đường xe bus V1

In [2]:
import time
from neo4j import GraphDatabase

# --- CẤU HÌNH ---
URI = "neo4j+s://403c0411.databases.neo4j.io"
AUTH = ("neo4j", "20qU7hUdIdlzyXii3tmnCDgvAFkUh-NobhE52Oq7Dvw") # <--- ĐIỀN PASS VÀO ĐÂY

class BusRAGBot:
    def __init__(self):
        self.driver = GraphDatabase.driver(URI, auth=AUTH)

    def close(self):
        self.driver.close()

    # --- PHẦN 1: RETRIEVAL (TRUY XUẤT DỮ LIỆU TỪ KG) ---
    # Đây là lúc RAG hoạt động: Thay vì tìm keyword trong văn bản, 
    # ta "Retrieve" (Truy xuất) đường đi chính xác từ Graph.
    def find_path_kg(self, start_query, end_query):
        with self.driver.session() as session:
            print(f"   🔍 [KG Retrieval] Đang tìm node trên Graph: '{start_query}' -> '{end_query}'...")
            
            # 1. Tìm Node (Entity Linking) - Ánh xạ từ ngữ người dùng vào Node trong Graph
            node_q = "MATCH (n:BusStop) WHERE toLower(n.name) CONTAINS toLower($name) RETURN n.name, n.id LIMIT 1"
            start_node = session.run(node_q, name=start_query).single()
            end_node = session.run(node_q, name=end_query).single()

            if not start_node: return f"❌ Không tìm thấy trạm nào tên '{start_query}' trong KG."
            if not end_node: return f"❌ Không tìm thấy trạm nào tên '{end_query}' trong KG."

            real_start, id_start = start_node['n.name'], start_node['n.id']
            real_end, id_end = end_node['n.name'], end_node['n.id']

            print(f"      ✅ Đã khớp Entity: {real_start} -> {real_end}")

            # 2. Thuật toán Graph (Reasoning)
            # Dùng shortestPath để tìm mối liên hệ giữa 2 Entity
            query = """
            MATCH (start:BusStop {id: $id_s}), (end:BusStop {id: $id_e})
            MATCH p = shortestPath((start)-[:NEXT_HOP*..60]->(end))
            WITH p, relationships(p) AS rels, nodes(p) AS stops
            RETURN [r IN rels | r.route_id] AS route_ids,
                   [s IN stops | s.name] AS stop_names,
                   reduce(t=0, r IN rels | t + r.time) AS total_time
            """
            result = session.run(query, id_s=id_start, id_e=id_end).single()

            if not result:
                return "❌ Không tìm thấy đường đi trong Knowledge Graph."

            # --- PHẦN 2: AUGMENTATION (LÀM GIÀU DỮ LIỆU) ---
            # Lấy thêm thông tin bổ trợ (Tên xe, Giá vé) để câu trả lời hay hơn
            route_ids = result['route_ids']
            stop_names = result['stop_names']
            total_time = result['total_time']
            
            unique_rids = list(set(route_ids))
            route_map = {}
            if unique_rids:
                # SỬA LỖI .list() TẠI ĐÂY: Dùng list(session.run(...))
                r_info = list(session.run("MATCH (r:BusRoute) WHERE r.id IN $rids RETURN r.id, r.route_no, r.price", rids=unique_rids))
                for r in r_info:
                    route_map[r['r.id']] = {'no': r['r.route_no'], 'price': r['r.price']}

            # --- PHẦN 3: GENERATION (TẠO CÂU TRẢ LỜI) ---
            # Ở đây chúng ta dùng Rule-based Generation (nếu dùng GPT sẽ là LLM Generation)
            response = ["\n" + "="*40]
            response.append(f"🚌 **LỘ TRÌNH TỐI ƯU TỪ KG** (Khoảng {int(total_time)} phút)")
            response.append("="*40)
            
            current_route_id = route_ids[0]
            start_segment = stop_names[0]
            
            for i in range(len(route_ids)):
                if route_ids[i] != current_route_id:
                    # Chuyển tuyến
                    r_data = route_map.get(current_route_id, {})
                    r_no = r_data.get('no', '?')
                    response.append(f"1️⃣ Đi xe **{r_no}** từ '{start_segment}' đến '{stop_names[i]}'.")
                    response.append(f"   ⬇ (Xuống trạm, chờ chuyển xe)")
                    current_route_id = route_ids[i]
                    start_segment = stop_names[i]
            
            # Chặng cuối
            r_data = route_map.get(current_route_id, {})
            r_no = r_data.get('no', '?')
            price = r_data.get('price', '').split('<br/>')[0].replace('- ', '')
            response.append(f"👉 Đi xe **{r_no}** từ '{start_segment}' đến đích '{stop_names[-1]}'.")
            if price: response.append(f"   💵 Giá vé: {price}")
            
            return "\n".join(response)

    # XỬ LÝ CÂU TỰ NHIÊN (NLP Đơn giản)
    def chat(self, user_input):
        user_input = user_input.lower()
        
        # Logic phân tích cú pháp: "đi từ [A] đến [B]"
        if "đến" in user_input:
            try:
                parts = user_input.split("đến")
                # Phần Điểm đi: Xóa các từ thừa "đi từ", "tìm đường"
                start = parts[0].replace("đi từ", "").replace("tìm đường", "").replace("cho tôi", "").strip()
                # Phần Điểm đến
                end = parts[1].strip()
                
                if not start or not end:
                    return "🤖 Bot: Vui lòng nhập rõ 'Đi từ A đến B'."
                
                return self.find_path_kg(start, end)
            except Exception as e:
                return f"⚠️ Lỗi xử lý câu: {e}"
        else:
            return "🤖 Bot: Bạn muốn đi đâu? Hãy nhập 'Đi từ [A] đến [B]'."

# --- CHẠY ---
if __name__ == "__main__":
    bot = BusRAGBot()
    print("\n👋 CHÀO BẠN! TÔI LÀ AI BUS MAP.")
    print("💡 Ví dụ: 'Đi từ đại học cảnh sát nhân dân đến nhà thờ đức bà'")
    
    while True:
        q = input("\n👤 Bạn: ")
        if q.lower() in ['exit', 'thoát']: break
        
        answer = bot.chat(q)
        print(answer)
    
    bot.close()


👋 CHÀO BẠN! TÔI LÀ AI BUS MAP.
💡 Ví dụ: 'Đi từ đại học cảnh sát nhân dân đến nhà thờ đức bà'


   🔍 [KG Retrieval] Đang tìm node trên Graph: 'đại học cảnh sát nhân dân' -> 'nhà thờ đức bà'...
❌ Không tìm thấy trạm nào tên 'đại học cảnh sát nhân dân' trong KG.
🤖 Bot: Bạn muốn đi đâu? Hãy nhập 'Đi từ [A] đến [B]'.
   🔍 [KG Retrieval] Đang tìm node trên Graph: 'đại học tôn đức thắng' -> 'nhà thờ đức bà'...
      ✅ Đã khớp Entity: Đại học Tôn Đức Thắng -> Nhà Thờ Đức Bà

🚌 **LỘ TRÌNH TỐI ƯU TỪ KG** (Khoảng 11 phút)
1️⃣ Đi xe **31** từ 'Đại học Tôn Đức Thắng' đến 'Đại học Cảnh sát'.
   ⬇ (Xuống trạm, chờ chuyển xe)
1️⃣ Đi xe **68** từ 'Đại học Cảnh sát' đến 'Siêu thị Lotte'.
   ⬇ (Xuống trạm, chờ chuyển xe)
1️⃣ Đi xe **140** từ 'Siêu thị Lotte' đến 'Cầu Kinh Tẻ'.
   ⬇ (Xuống trạm, chờ chuyển xe)
1️⃣ Đi xe **139** từ 'Cầu Kinh Tẻ' đến 'Cầu Ông Lãnh'.
   ⬇ (Xuống trạm, chờ chuyển xe)
1️⃣ Đi xe **31** từ 'Cầu Ông Lãnh' đến 'Trạm Trung chuyển trên đường Hàm Nghi'.
   ⬇ (Xuống trạm, chờ chuyển xe)
1️⃣ Đi xe **44** từ 'Trạm Trung chuyển trên đường Hàm Nghi' đến 'Chùa Bà Ấn'.
   ⬇ (Xuống tr

### => Vấn đề gặp phải
1. Vấn đề Tìm tên (Entity Linking): ở trên dùng CONTAINS (chứa chuỗi). Khi gõ "Đại học Cảnh sát nhân dân", nhưng trong DB chỉ có "Đại học Cảnh sát". Chuỗi ngắn không thể chứa chuỗi dài -> Tìm không ra.

2. Vấn đề Lộ trình (Routing): Thuật toán shortestPath mặc định của Neo4j tìm đường đi qua ÍT TRẠM NHẤT (về mặt hình học), chứ không quan tâm việc CHUYỂN TUYẾN (Transfer penalty). Nó thà nhảy qua 7 xe buýt để đi 10 trạm còn hơn đi 1 xe buýt mà phải qua 15 trạm.

### Cải tiến
1. Fulltext Search (Lucene): Để tìm kiếm mờ (Fuzzy), chấp nhận gõ sai hoặc thừa từ.

2. Smart Routing Strategy (Chiến thuật Thông minh): Thay vì tìm đường mù quáng, ta ưu tiên theo thứ tự: Đi thẳng (0 transfer) -> Chuyển 1 lần -> Mới dùng Graph (cho ca khó).

### Test tìm bus cải tiến V2

In [3]:
import time
from neo4j import GraphDatabase

# --- CẤU HÌNH ---
URI = "neo4j+s://403c0411.databases.neo4j.io"
AUTH = ("neo4j", "20qU7hUdIdlzyXii3tmnCDgvAFkUh-NobhE52Oq7Dvw")

class BusProBot:
    def __init__(self):
        self.driver = GraphDatabase.driver(URI, auth=AUTH)

    def close(self):
        self.driver.close()

    # --- 1. TÌM TRẠM THÔNG MINH (FULLTEXT SEARCH) ---
    def find_stop_node(self, session, query_text):
        # Dùng Lucene Query để tìm kiếm mờ (Fuzzy ~)
        # Ví dụ: "Đại học Cảnh sát nhân dân" vẫn khớp "Đại học Cảnh sát"
        clean_query = query_text.replace("trạm", "").strip()
        
        # Thử tìm chính xác trước
        q_exact = "MATCH (n:BusStop) WHERE toLower(n.name) CONTAINS toLower($txt) RETURN n LIMIT 1"
        res = session.run(q_exact, txt=clean_query).single()
        
        if res: return res['n']

        # Nếu không ra, dùng Fulltext Search (Mạnh mẽ hơn)
        try:
            # Query mờ: thêm dấu ~ vào cuối các từ
            lucene_query = " OR ".join([f"{word}~" for word in clean_query.split()])
            q_full = f"""
            CALL db.index.fulltext.queryNodes("stopNameIndex", '{lucene_query}') YIELD node, score
            RETURN node ORDER BY score DESC LIMIT 1
            """
            res = session.run(q_full).single()
            if res: return res['node']
        except:
            pass
            
        return None

    # --- 2. CHIẾN THUẬT TÌM ĐƯỜNG (ROUTING STRATEGY) ---
    def find_best_route(self, start_text, end_text):
        with self.driver.session() as session:
            # A. Tìm Node
            print(f"   🔍 Đang tìm trạm: '{start_text}' -> '{end_text}'...")
            s_node = self.find_stop_node(session, start_text)
            e_node = self.find_stop_node(session, end_text)

            if not s_node: return f"❌ Không tìm thấy trạm nào khớp với '{start_text}'"
            if not e_node: return f"❌ Không tìm thấy trạm nào khớp với '{end_text}'"

            s_id, s_name = s_node['id'], s_node['name']
            e_id, e_name = e_node['id'], e_node['name']
            
            print(f"      ✅ Đã khớp: {s_name} ({s_id}) -> {e_name} ({e_id})")

            # B. CHIẾN THUẬT 1: TÌM TUYẾN ĐI THẲNG (Ưu tiên số 1)
            # Kiểm tra xem 2 trạm có cùng thuộc 1 tuyến, cùng chiều và thứ tự hợp lý không
            q_direct = """
            MATCH (s:BusStop {id: $sid})-[r1:ON_ROUTE]->(route:BusRoute)<-[r2:ON_ROUTE]-(e:BusStop {id: $eid})
            WHERE r1.direction = r2.direction 
              AND r1.order < r2.order
            RETURN route.route_no AS bus_no, route.name AS route_name, 
                   (r2.order - r1.order) AS stops_count, r1.direction AS dir
            ORDER BY stops_count ASC LIMIT 1
            """
            direct = session.run(q_direct, sid=s_id, eid=e_id).single()

            if direct:
                return (f"\n🎯 **TÌM THẤY TUYẾN ĐI THẲNG!** (Không cần chuyển xe)\n"
                        f"   🚌 **Đi xe {direct['bus_no']}**: {direct['route_name']}\n"
                        f"   📍 Từ: {s_name}\n"
                        f"   🏁 Đến: {e_name}\n"
                        f"   ⏱ Qua khoảng {direct['stops_count']} trạm dừng.")

            # C. CHIẾN THUẬT 2: TÌM 1 LẦN CHUYỂN XE (Ưu tiên số 2)
            # Tìm trạm trung chuyển X mà: (Start -> X) chung tuyến A VÀ (X -> End) chung tuyến B
            print("   ⚠️ Không có tuyến đi thẳng. Đang tìm lộ trình chuyển 1 lần...")
            q_transfer = """
            MATCH (s:BusStop {id: $sid})-[r1:ON_ROUTE]->(route1:BusRoute)<-[r2:ON_ROUTE]-(transfer:BusStop)
            WHERE r1.direction = r2.direction AND r1.order < r2.order
            
            MATCH (transfer)-[r3:ON_ROUTE]->(route2:BusRoute)<-[r4:ON_ROUTE]-(e:BusStop {id: $eid})
            WHERE r3.direction = r4.direction AND r3.order < r4.order AND route1 <> route2
            
            RETURN route1.route_no AS bus1, route2.route_no AS bus2, transfer.name AS transfer_stop
            LIMIT 1
            """
            one_stop = session.run(q_transfer, sid=s_id, eid=e_id).single()

            if one_stop:
                return (f"\n🔄 **LỘ TRÌNH 1 LẦN CHUYỂN XE**\n"
                        f"   1️⃣ Đi xe **{one_stop['bus1']}** từ '{s_name}' đến trạm '{one_stop['transfer_stop']}'.\n"
                        f"   ⬇ (Xuống xe, chờ chuyển tuyến)\n"
                        f"   2️⃣ Đi xe **{one_stop['bus2']}** từ '{one_stop['transfer_stop']}' đến đích '{e_name}'.")

            # D. CHIẾN THUẬT 3: GRAPH SHORTEST PATH (Phương án cuối cùng)
            # Nếu 2 cách trên không được, mới dùng thuật toán Graph (nhưng lọc bớt)
            print("   ⚠️ Lộ trình phức tạp. Đang dùng Graph Algorithm...")
            # (Giữ nguyên logic cũ nhưng cảnh báo người dùng)
            return "❌ Quá xa hoặc quá phức tạp, tôi chưa tìm ra đường tối ưu dưới 2 lần chuyển."

    def chat(self, text):
        if "đến" in text.lower():
            try:
                parts = text.lower().split("đến")
                start = parts[0].replace("đi từ", "").replace("tìm đường", "").strip()
                end = parts[1].strip()
                return self.find_best_route(start, end)
            except:
                return "Lỗi cú pháp."
        return "Hãy nhập: 'Đi từ A đến B'"

if __name__ == "__main__":
    bot = BusProBot()
    print("\n👋 BUS MAP PRO: Ưu tiên đi thẳng & Ít chuyển tuyến!")
    
    while True:
        q = input("\n👤 Bạn: ")
        if q.lower() == 'exit': break
        print(bot.chat(q))


👋 BUS MAP PRO: Ưu tiên đi thẳng & Ít chuyển tuyến!
   🔍 Đang tìm trạm: 'đh csnd' -> 'chợ bên thành'...
      ✅ Đã khớp: Đại học CSND (4471) -> Ga Bến Thành (7003)
   ⚠️ Không có tuyến đi thẳng. Đang tìm lộ trình chuyển 1 lần...
   ⚠️ Lộ trình phức tạp. Đang dùng Graph Algorithm...
❌ Quá xa hoặc quá phức tạp, tôi chưa tìm ra đường tối ưu dưới 2 lần chuyển.
   🔍 Đang tìm trạm: 'đh tdt' -> 'đại học văn lang'...
      ✅ Đã khớp: ĐH Nguyễn Tất Thành (2361) -> Trường Đại học Văn Lang (7597)
   ⚠️ Không có tuyến đi thẳng. Đang tìm lộ trình chuyển 1 lần...
   ⚠️ Lộ trình phức tạp. Đang dùng Graph Algorithm...
❌ Quá xa hoặc quá phức tạp, tôi chưa tìm ra đường tối ưu dưới 2 lần chuyển.
   🔍 Đang tìm trạm: 'đh tôn đức thắng' -> 'đh văn lang'...
      ✅ Đã khớp: ĐH Tôn Đức Thắng (1200) -> Bến xe Ngã 3 Tân Lân (7220)
   ⚠️ Không có tuyến đi thẳng. Đang tìm lộ trình chuyển 1 lần...
   ⚠️ Lộ trình phức tạp. Đang dùng Graph Algorithm...
❌ Quá xa hoặc quá phức tạp, tôi chưa tìm ra đường tối ưu dưới 2 

### Test bus V3

In [4]:
import time
import re
from neo4j import GraphDatabase

# --- CẤU HÌNH ---
URI = "neo4j+s://403c0411.databases.neo4j.io"
AUTH = ("neo4j", "20qU7hUdIdlzyXii3tmnCDgvAFkUh-NobhE52Oq7Dvw")

class BusBotV4:
    def __init__(self):
        self.driver = GraphDatabase.driver(URI, auth=AUTH)
        
        # TỪ ĐIỂN VIẾT TẮT (Ưu tiên tuyệt đối)
        self.ABBREVIATIONS = {
            "tdt": "đại học tôn đức thắng",
            "đh tdt": "đại học tôn đức thắng",
            "csnd": "đại học cảnh sát nhân dân",
            "đh csnd": "đại học cảnh sát nhân dân",
            "đh": "đại học",
            "kdc": "khu dân cư",
            "bx": "bến xe",
            "bến thành": "bến xe buýt sài gòn", # Chuẩn hóa tên Bến Thành
            "chợ bến thành": "bến xe buýt sài gòn"
        }

    def close(self):
        self.driver.close()

    def normalize_query(self, text):
        """Chuẩn hóa câu hỏi, xử lý từ viết tắt"""
        text = text.lower()
        # Thay thế các từ khóa đặc biệt
        for abbr, full in self.ABBREVIATIONS.items():
            # Dùng Regex để thay thế chính xác từ (tránh thay nhầm chữ con)
            pattern = r'\b' + re.escape(abbr) + r'\b'
            text = re.sub(pattern, full, text)
        return text

    def find_stop_node(self, session, query_text):
        clean_query = self.normalize_query(query_text).replace("trạm", "").strip()
        print(f"      Debugging: '{query_text}' -> '{clean_query}'")

        # 1. Tìm CHÍNH XÁC (Ưu tiên số 1)
        # Tìm trong Name hoặc Search (viết tắt không dấu)
        q_exact = """
        MATCH (n:BusStop) 
        WHERE toLower(n.name) CONTAINS $txt 
           OR toLower(n.search) CONTAINS $txt
        RETURN n ORDER BY size(n.name) ASC LIMIT 1
        """
        res = session.run(q_exact, txt=clean_query).single()
        if res: return res['n']

        # 2. Tìm FULLTEXT (Nếu chính xác không ra)
        try:
            lucene_query = " AND ".join([f"{w}~" for w in clean_query.split()])
            q_full = f"""
            CALL db.index.fulltext.queryNodes("stopNameIndex", '{lucene_query}') YIELD node, score
            RETURN node ORDER BY score DESC LIMIT 1
            """
            res = session.run(q_full).single()
            if res: return res['node']
        except: pass
            
        return None

    def find_path_any_cost(self, start_text, end_text):
        with self.driver.session() as session:
            # A. TÌM TRẠM
            print(f"   🔍 Đang tìm trạm: '{start_text}' -> '{end_text}'...")
            s_node = self.find_stop_node(session, start_text)
            e_node = self.find_stop_node(session, end_text)

            if not s_node: return f"❌ Không tìm thấy trạm đi khớp với '{start_text}'"
            if not e_node: return f"❌ Không tìm thấy trạm đến khớp với '{end_text}'"

            s_id, s_name = s_node['id'], s_node['name']
            e_id, e_name = e_node['id'], e_node['name']
            
            print(f"      ✅ Đã khớp: {s_name} -> {e_name}")

            # B. CẤP ĐỘ 1: ĐI THẲNG (Nới lỏng điều kiện)
            # Chỉ cần cùng tuyến và order đi < order đến (bỏ qua check direction quá gắt)
            q_direct = """
            MATCH (s:BusStop {id: $sid})-[r1:ON_ROUTE]->(route:BusRoute)<-[r2:ON_ROUTE]-(e:BusStop {id: $eid})
            WHERE r1.order < r2.order
            RETURN route.route_no AS bus_no, route.name AS route_name, 
                   (r2.order - r1.order) AS stops_count
            ORDER BY stops_count ASC LIMIT 1
            """
            direct = session.run(q_direct, sid=s_id, eid=e_id).single()

            if direct:
                return (f"\n🎯 **TÌM THẤY TUYẾN ĐI THẲNG!**\n"
                        f"   🚌 **Đi xe {direct['bus_no']}**: {direct['route_name']}\n"
                        f"   📍 Từ: {s_name}\n"
                        f"   🏁 Đến: {e_name}\n"
                        f"   ⏱ Qua khoảng {direct['stops_count']} trạm.")

            # C. CẤP ĐỘ 2: GRAPH ALGORITHM (TÌM MỌI CÁCH)
            # Nếu không đi thẳng được, dùng shortestPath tìm đường ngắn nhất bất kể bao nhiêu lần đổi
            print("   ⚠️ Không có xe đi thẳng. Đang tính toán lộ trình tối ưu nhất (Graph)...")
            
            q_graph = """
            MATCH (start:BusStop {id: $sid}), (end:BusStop {id: $eid})
            MATCH p = shortestPath((start)-[:NEXT_HOP*..100]->(end))
            
            // Phân tích lộ trình
            WITH p, relationships(p) AS rels, nodes(p) AS stops
            RETURN [r IN rels | r.route_id] AS route_ids,
                   [s IN stops | s.name] AS stop_names,
                   reduce(t=0, r IN rels | t + r.time) AS total_time,
                   length(p) AS hops
            """
            result = session.run(q_graph, sid=s_id, eid=e_id).single()

            if not result:
                return "❌ Không tìm thấy đường đi nào (2 trạm có thể bị cô lập)."

            # --- DỰNG LẠI LỘ TRÌNH TỪ KẾT QUẢ GRAPH ---
            route_ids = result['route_ids']
            stop_names = result['stop_names']
            total_time = result['total_time']
            
            # Lấy thông tin số hiệu xe
            unique_rids = list(set(route_ids))
            route_map = {}
            if unique_rids:
                r_info = list(session.run("MATCH (r:BusRoute) WHERE r.id IN $rids RETURN r.id, r.route_no", rids=unique_rids))
                for r in r_info:
                    route_map[r['r.id']] = r['r.route_no']

            response = ["\n" + "="*40]
            response.append(f"🔄 **LỘ TRÌNH CHUYỂN TUYẾN** (Khoảng {int(total_time)} phút)")
            response.append("="*40)
            
            current_rid = route_ids[0]
            start_seg = stop_names[0]
            
            step = 1
            for i in range(len(route_ids)):
                if route_ids[i] != current_rid:
                    # Đổi xe
                    bus_no = route_map.get(current_rid, "???")
                    response.append(f"{step}. Đi xe **{bus_no}** từ '{start_seg}' đến '{stop_names[i]}'.")
                    response.append(f"   ⬇ (Xuống trạm, đổi xe)")
                    current_rid = route_ids[i]
                    start_seg = stop_names[i]
                    step += 1
            
            # Chặng cuối
            bus_no = route_map.get(current_rid, "???")
            response.append(f"{step}. Đi xe **{bus_no}** từ '{start_seg}' đến đích '{stop_names[-1]}'.")
            
            return "\n".join(response)

    def chat(self, text):
        if "đến" in text.lower():
            try:
                parts = text.lower().split("đến")
                start = parts[0].replace("đi từ", "").replace("tìm đường", "").strip()
                end = parts[1].strip()
                return self.find_path_any_cost(start, end)
            except Exception as e:
                return f"Lỗi: {e}"
        return "Hãy nhập: 'Đi từ [A] đến [B]'"

if __name__ == "__main__":
    bot = BusBotV4()
    print("\n👋 BUS MAP V4: Fix lỗi nhận diện & Tìm mọi đường!")
    
    while True:
        q = input("\n👤 Bạn: ")
        if q.lower() in ['exit', 'quit']: break
        print(bot.chat(q))


👋 BUS MAP V4: Fix lỗi nhận diện & Tìm mọi đường!
   🔍 Đang tìm trạm: 'đh tdt' -> 'đh văn lang'...
      Debugging: 'đh tdt' -> 'đại học đại học tôn đức thắng'
      Debugging: 'đh văn lang' -> 'đại học văn lang'
      ✅ Đã khớp: Đại học Tôn Đức Thắng -> Trường Đại học Văn Lang
   ⚠️ Không có xe đi thẳng. Đang tính toán lộ trình tối ưu nhất (Graph)...

🔄 **LỘ TRÌNH CHUYỂN TUYẾN** (Khoảng 25 phút)
1. Đi xe **31** từ 'Đại học Tôn Đức Thắng' đến 'Đại học Cảnh sát'.
   ⬇ (Xuống trạm, đổi xe)
2. Đi xe **68** từ 'Đại học Cảnh sát' đến 'Siêu thị Lotte'.
   ⬇ (Xuống trạm, đổi xe)
3. Đi xe **140** từ 'Siêu thị Lotte' đến 'Cầu Kinh Tẻ'.
   ⬇ (Xuống trạm, đổi xe)
4. Đi xe **139** từ 'Cầu Kinh Tẻ' đến 'Cầu Ông Lãnh'.
   ⬇ (Xuống trạm, đổi xe)
5. Đi xe **31** từ 'Cầu Ông Lãnh' đến 'Nguyễn Kim'.
   ⬇ (Xuống trạm, đổi xe)
6. Đi xe **75** từ 'Nguyễn Kim' đến 'Trạm Trung chuyển trên đường Hàm Nghi'.
   ⬇ (Xuống trạm, đổi xe)
7. Đi xe **DL01** từ 'Trạm Trung chuyển trên đường Hàm Nghi' đến 'Hồ Tùng Mậu'

### Test bus V4

In [6]:
import time
import re
from neo4j import GraphDatabase

# --- CẤU HÌNH ---
URI = "neo4j+s://403c0411.databases.neo4j.io"
AUTH = ("neo4j", "20qU7hUdIdlzyXii3tmnCDgvAFkUh-NobhE52Oq7Dvw")

class BusBotUniversal:
    def __init__(self):
        self.driver = GraphDatabase.driver(URI, auth=AUTH)
        
        # TỪ ĐIỂN ĐA NĂNG (Thêm các địa điểm hot ở HCM)
        self.ABBREVIATIONS = {
            # Trường học
            "tdt": "đại học tôn đức thắng",
            "đh tdt": "đại học tôn đức thắng",
            "csnd": "đại học cảnh sát nhân dân",
            "đh csnd": "đại học cảnh sát nhân dân",
            "đh": "đại học",
            "văn lang": "trường đại học văn lang",
            "đh văn lang": "trường đại học văn lang",
            "bách khoa": "đại học bách khoa",
            "khtn": "khoa học tự nhiên",
            "spkt": "sư phạm kỹ thuật",
            
            # Địa danh
            "kdc": "khu dân cư",
            "bx": "bến xe",
            "bến thành": "bến xe buýt sài gòn",
            "chợ bến thành": "bến xe buýt sài gòn",
            "miền đông": "bến xe miền đông",
            "miền tây": "bến xe miền tây",
            "suối tiên": "khu du lịch suối tiên",
            "đầm sen": "công viên văn hóa đầm sen",
            "chợ lớn": "bến xe buýt chợ lớn",
            "sân bay": "sân bay tân sơn nhất"
        }

    def close(self):
        self.driver.close()

    def normalize_query(self, text):
        text = text.lower()
        for abbr, full in self.ABBREVIATIONS.items():
            pattern = r'\b' + re.escape(abbr) + r'\b'
            text = re.sub(pattern, full, text)
        return text

    def find_stop_node(self, session, query_text):
        clean_query = self.normalize_query(query_text).replace("trạm", "").strip()
        
        # 1. Tìm CHÍNH XÁC (Ưu tiên số 1 - Nhanh & Chuẩn)
        q_exact = """
        MATCH (n:BusStop) 
        WHERE toLower(n.name) CONTAINS $txt 
           OR toLower(n.search) CONTAINS $txt
        RETURN n ORDER BY size(n.name) ASC LIMIT 1
        """
        res = session.run(q_exact, txt=clean_query).single()
        if res: return res['n']

        # 2. Tìm FULLTEXT (Nếu người dùng gõ sai chính tả)
        try:
            # Tạo query fuzzy (~) cho từng từ
            lucene_query = " AND ".join([f"{w}~" for w in clean_query.split()])
            q_full = f"""
            CALL db.index.fulltext.queryNodes("stopNameIndex", '{lucene_query}') YIELD node, score
            RETURN node ORDER BY score DESC LIMIT 1
            """
            res = session.run(q_full).single()
            if res: return res['node']
        except: pass
        return None

    def solve_route(self, start_text, end_text):
        with self.driver.session() as session:
            # A. NHẬN DIỆN ĐIỂM ĐI/ĐẾN
            print(f"   🔍 Phân tích: '{start_text}' -> '{end_text}'...")
            s_node = self.find_stop_node(session, start_text)
            e_node = self.find_stop_node(session, end_text)

            if not s_node: return f"❌ Bot chưa biết trạm '{start_text}' ở đâu. Bạn thử nhập tên đầy đủ xem?"
            if not e_node: return f"❌ Bot chưa biết trạm '{end_text}' ở đâu. Bạn thử nhập tên đầy đủ xem?"

            s_id, s_name = s_node['id'], s_node['name']
            e_id, e_name = e_node['id'], e_node['name']
            print(f"      ✅ Đã hiểu: {s_name} -> {e_name}")

            # B. CHIẾN THUẬT 1: TÌM XE ĐI THẲNG (Direct)
            # Logic: Tìm xe X đi qua cả A và B, và A nằm trước B
            q_direct = """
            MATCH (s:BusStop {id: $sid})-[r1:ON_ROUTE]->(route:BusRoute)<-[r2:ON_ROUTE]-(e:BusStop {id: $eid})
            WHERE r1.order < r2.order
            RETURN route.route_no AS bus_no, route.name AS route_name, 
                   (r2.order - r1.order) AS stops_count
            ORDER BY stops_count ASC LIMIT 1
            """
            direct = session.run(q_direct, sid=s_id, eid=e_id).single()

            if direct:
                return (f"\n🎯 **CÓ XE ĐI THẲNG NÈ!**\n"
                        f"   🚌 **Đi xe {direct['bus_no']}**: {direct['route_name']}\n"
                        f"   📍 Đón tại: {s_name}\n"
                        f"   🏁 Xuống tại: {e_name}\n"
                        f"   ⏱ Lộ trình qua {direct['stops_count']} trạm dừng.")

            # C. CHIẾN THUẬT 2: TÌM 1 TRẠM TRUNG CHUYỂN
            # Logic: A -> [Xe 1] -> Trạm X -> [Xe 2] -> B
            print("      ⚠️ Không có xe đi thẳng. Đang tìm trạm trung chuyển tối ưu...")
            
            q_transfer = """
            MATCH (s:BusStop {id: $sid})-[r1:ON_ROUTE]->(route1:BusRoute)<-[r2:ON_ROUTE]-(transfer:BusStop)
            WHERE r1.order < r2.order
            
            MATCH (transfer)-[r3:ON_ROUTE]->(route2:BusRoute)<-[r4:ON_ROUTE]-(e:BusStop {id: $eid})
            WHERE r3.order < r4.order AND route1 <> route2
            
            RETURN route1.route_no AS bus1, route2.route_no AS bus2, transfer.name AS transfer_stop
            LIMIT 1
            """
            one_stop = session.run(q_transfer, sid=s_id, eid=e_id).single()

            if one_stop:
                return (f"\n🔄 **LỘ TRÌNH 1 LẦN ĐỔI XE**\n"
                        f"   1️⃣ Đi xe **{one_stop['bus1']}** từ '{s_name}' đến trạm '{one_stop['transfer_stop']}'.\n"
                        f"   ⬇ (Xuống xe, chờ một chút)\n"
                        f"   2️⃣ Đón tiếp xe **{one_stop['bus2']}** đi tới '{e_name}'.")

            # D. CHIẾN THUẬT 3: TÌM MỌI CÁCH (Graph Search)
            # Dành cho các ca khó (Suối Tiên -> Củ Chi...)
            print("      ⚠️ Lộ trình xa. Đang quét toàn bộ mạng lưới...")
            q_graph = """
            MATCH (start:BusStop {id: $sid}), (end:BusStop {id: $eid})
            MATCH p = shortestPath((start)-[:NEXT_HOP*..150]->(end)) 
            
            WITH p, relationships(p) AS rels, nodes(p) AS stops
            RETURN [r IN rels | r.route_id] AS route_ids,
                   [s IN stops | s.name] AS stop_names,
                   reduce(t=0, r IN rels | t + r.time) AS total_time
            """
            result = session.run(q_graph, sid=s_id, eid=e_id).single()

            if result:
                route_ids = result['route_ids']
                stop_names = result['stop_names']
                
                # Mapping ID -> Số xe
                unique_rids = list(set(route_ids))
                route_map = {}
                if unique_rids:
                    r_info = list(session.run("MATCH (r:BusRoute) WHERE r.id IN $rids RETURN r.id, r.route_no", rids=unique_rids))
                    for r in r_info: route_map[r['r.id']] = r['r.route_no']

                response = [f"\n🗺️ **LỘ TRÌNH PHỨC TẠP** (Khoảng {int(result['total_time'])} phút)"]
                
                current_rid = route_ids[0]
                start_seg = stop_names[0]
                step = 1
                
                for i in range(len(route_ids)):
                    if route_ids[i] != current_rid:
                        bus_no = route_map.get(current_rid, "?")
                        response.append(f"{step}. Đi xe **{bus_no}** từ '{start_seg}' đến '{stop_names[i]}'.")
                        current_rid = route_ids[i]
                        start_seg = stop_names[i]
                        step += 1
                
                bus_no = route_map.get(current_rid, "?")
                response.append(f"{step}. Đi xe **{bus_no}** từ '{start_seg}' đến đích '{stop_names[-1]}'.")
                return "\n".join(response)

            return "❌ Hai địa điểm này quá xa hoặc không có kết nối xe buýt."

    def chat(self, text):
        if "đến" in text.lower():
            try:
                parts = text.lower().split("đến")
                start = parts[0].replace("đi từ", "").replace("tìm đường", "").replace("cho tôi", "").strip()
                end = parts[1].strip()
                return self.solve_route(start, end)
            except:
                pass
        return "🤖 Bot: Hãy nhập 'Đi từ [A] đến [B]'"

if __name__ == "__main__":
    bot = BusBotUniversal()
    print("\n👋 HỆ THỐNG BUS THÔNG MINH (ALL ROUTES SUPPORTED)")
    print("💡 Thử nhập: 'Đi từ Bến Thành đến Suối Tiên' xem!")
    
    while True:
        q = input("\n👤 Bạn: ")
        if q.lower() in ['exit', 'quit']: break
        print(bot.chat(q))


👋 HỆ THỐNG BUS THÔNG MINH (ALL ROUTES SUPPORTED)
💡 Thử nhập: 'Đi từ Bến Thành đến Suối Tiên' xem!
   🔍 Phân tích: 'bến thành' -> 'suối tiên'...
❌ Bot chưa biết trạm 'suối tiên' ở đâu. Bạn thử nhập tên đầy đủ xem?
   🔍 Phân tích: 'đh tôn đức thắng' -> 'đh văn lang'...
      ✅ Đã hiểu: Đại học Tôn Đức Thắng -> Trường Đại học Văn Lang

🎯 **CÓ XE ĐI THẲNG NÈ!**
   🚌 **Đi xe 31**: Bus 31: Đại Học Tôn Đức Thắng - Bến Thành - Đại học Văn Lang
   📍 Đón tại: Đại học Tôn Đức Thắng
   🏁 Xuống tại: Trường Đại học Văn Lang
   ⏱ Lộ trình qua 56 trạm dừng.
   🔍 Phân tích: 'chợ bến thành' -> 'suối tiên'...
❌ Bot chưa biết trạm 'chợ bến thành' ở đâu. Bạn thử nhập tên đầy đủ xem?
   🔍 Phân tích: 'chợ bên thành' -> 'nhà thờ đức bà'...
      ✅ Đã hiểu: Chợ Bến Thành -> Nhà Thờ Đức Bà
      ⚠️ Không có xe đi thẳng. Đang tìm trạm trung chuyển tối ưu...
      ⚠️ Lộ trình xa. Đang quét toàn bộ mạng lưới...

🗺️ **LỘ TRÌNH PHỨC TẠP** (Khoảng 2 phút)
1. Đi xe **155** từ 'Chợ Bến Thành' đến 'Ga Bến Thành'.
2. Đi

### Test bus V5

In [8]:
import time
import re
import requests
from neo4j import GraphDatabase

# --- CẤU HÌNH ---
URI = "neo4j+s://403c0411.databases.neo4j.io"
AUTH = ("neo4j", "20qU7hUdIdlzyXii3tmnCDgvAFkUh-NobhE52Oq7Dvw")

class BusBotFinal:
    def __init__(self):
        self.driver = GraphDatabase.driver(URI, auth=AUTH)
        
        self.ABBREVIATIONS = {
            "tdt": "đại học tôn đức thắng",
            "đh tdt": "đại học tôn đức thắng",
            "csnd": "đại học cảnh sát nhân dân",
            "đh": "đại học",
            "văn lang": "trường đại học văn lang",
            "suối tiên": "khu du lịch suối tiên",
            "bến thành": "bến xe buýt sài gòn",
            "chợ bến thành": "bến xe buýt sài gòn",
            "nhà thờ đức bà": "công xã paris",
        }

    def close(self):
        self.driver.close()

    # --- GEOLOCATION (TÌM TỌA ĐỘ) ---
    def get_coordinates(self, place_name):
        try:
            url = "https://nominatim.openstreetmap.org/search"
            params = {'q': f"{place_name}, Ho Chi Minh City, Vietnam", 'format': 'json', 'limit': 1}
            headers = {'User-Agent': 'BusMapBot/1.0'}
            resp = requests.get(url, params=params, headers=headers, timeout=3)
            data = resp.json()
            if data: return float(data[0]['lat']), float(data[0]['lon']), data[0]['display_name']
        except: pass
        return None, None, None

    # --- TÌM TRẠM GẦN NHẤT ---
    def find_nearest_station_by_coords(self, session, lat, lng):
        query = """
        MATCH (n:BusStop)
        WITH n, point.distance(point({latitude: n.lat, longitude: n.lng}), point({latitude: $lat, longitude: $lng})) AS dist
        WHERE dist < 1000 
        RETURN n, dist ORDER BY dist ASC LIMIT 1
        """
        result = session.run(query, lat=lat, lng=lng).single()
        if result: return result['n'], result['dist']
        return None, None

    def normalize_query(self, text):
        text = text.lower()
        for abbr, full in self.ABBREVIATIONS.items():
            pattern = r'\b' + re.escape(abbr) + r'\b'
            text = re.sub(pattern, full, text)
        return text

    def find_stop_node(self, session, query_text):
        clean_query = self.normalize_query(query_text).replace("trạm", "").strip()
        print(f"      🔎 Đang tìm trạm cho: '{clean_query}'...")

        # 1. Chính xác
        q_exact = "MATCH (n:BusStop) WHERE toLower(n.name) CONTAINS $txt OR toLower(n.search) CONTAINS $txt RETURN n ORDER BY size(n.name) ASC LIMIT 1"
        res = session.run(q_exact, txt=clean_query).single()
        if res: return res['n']

        # 2. Fulltext
        try:
            lucene_query = " AND ".join([f"{w}~" for w in clean_query.split()])
            q_full = f"CALL db.index.fulltext.queryNodes('stopNameIndex', '{lucene_query}') YIELD node, score RETURN node ORDER BY score DESC LIMIT 1"
            res = session.run(q_full).single()
            if res: return res['node']
        except: pass

        # 3. Geolocation
        print(f"      🌍 Không thấy tên trạm. Đang tìm địa điểm trên bản đồ số...")
        lat, lng, address = self.get_coordinates(clean_query)
        if lat and lng:
            print(f"         📍 Tìm thấy địa điểm: {address[:40]}...")
            nearest_node, dist = self.find_nearest_station_by_coords(session, lat, lng)
            if nearest_node:
                print(f"         🚌 Trạm gần nhất ({int(dist)}m): {nearest_node['name']}")
                return nearest_node
        return None

    def solve_route(self, start_text, end_text):
        with self.driver.session() as session:
            s_node = self.find_stop_node(session, start_text)
            e_node = self.find_stop_node(session, end_text)

            if not s_node: return f"❌ Bó tay. Không tìm thấy địa điểm '{start_text}'."
            if not e_node: return f"❌ Bó tay. Không tìm thấy địa điểm '{end_text}'."

            s_id, s_name = s_node['id'], s_node['name']
            e_id, e_name = e_node['id'], e_node['name']
            print(f"      ✅ Chốt lộ trình: {s_name} -> {e_name}")

            # 1. ĐI THẲNG
            q_direct = """
            MATCH (s:BusStop {id: $sid})-[r1:ON_ROUTE]->(route:BusRoute)<-[r2:ON_ROUTE]-(e:BusStop {id: $eid})
            WHERE r1.order < r2.order
            RETURN route.route_no AS bus_no, route.name AS route_name, (r2.order - r1.order) AS stops_count
            ORDER BY stops_count ASC LIMIT 1
            """
            direct = session.run(q_direct, sid=s_id, eid=e_id).single()
            if direct:
                return (f"\n🎯 **CÓ XE ĐI THẲNG NÈ!**\n   🚌 **Đi xe {direct['bus_no']}**: {direct['route_name']}\n   📍 Từ: {s_name}\n   🏁 Đến: {e_name}\n   ⏱ Qua: {direct['stops_count']} trạm dừng.")

            # 2. CHUYỂN 1 LẦN
            q_transfer = """
            MATCH (s:BusStop {id: $sid})-[r1:ON_ROUTE]->(route1:BusRoute)<-[r2:ON_ROUTE]-(transfer:BusStop)
            WHERE r1.order < r2.order
            MATCH (transfer)-[r3:ON_ROUTE]->(route2:BusRoute)<-[r4:ON_ROUTE]-(e:BusStop {id: $eid})
            WHERE r3.order < r4.order AND route1 <> route2
            RETURN route1.route_no AS bus1, route2.route_no AS bus2, transfer.name AS transfer_stop LIMIT 1
            """
            one_stop = session.run(q_transfer, sid=s_id, eid=e_id).single()
            if one_stop:
                return (f"\n🔄 **LỘ TRÌNH 1 LẦN ĐỔI XE**\n   1️⃣ Đi xe **{one_stop['bus1']}** từ '{s_name}' đến trạm '{one_stop['transfer_stop']}'.\n   ⬇ (Xuống xe, chờ xíu)\n   2️⃣ Đón tiếp xe **{one_stop['bus2']}** đi tới '{e_name}'.")

            # 3. GRAPH (PHỨC TẠP) - ĐÃ BỔ SUNG CODE HIỂN THỊ ĐẦY ĐỦ
            q_graph = """
            MATCH (start:BusStop {id: $sid}), (end:BusStop {id: $eid})
            MATCH p = shortestPath((start)-[:NEXT_HOP*..150]->(end)) 
            WITH p, relationships(p) AS rels, nodes(p) AS stops
            RETURN [r IN rels | r.route_id] AS route_ids, [s IN stops | s.name] AS stop_names, reduce(t=0, r IN rels | t + r.time) AS total_time
            """
            result = session.run(q_graph, sid=s_id, eid=e_id).single()

            if result:
                # --- PHẦN CODE BỊ THIẾU LÚC NÃY ĐÃ ĐƯỢC THÊM VÀO ĐÂY ---
                route_ids = result['route_ids']
                stop_names = result['stop_names']
                total_time = result['total_time']

                # Lấy tên xe từ ID
                unique_rids = list(set(route_ids))
                route_map = {}
                if unique_rids:
                    r_info = list(session.run("MATCH (r:BusRoute) WHERE r.id IN $rids RETURN r.id, r.route_no", rids=unique_rids))
                    for r in r_info: route_map[r['r.id']] = r['r.route_no']

                # Tạo chuỗi kết quả chi tiết
                response = ["\n" + "="*30]
                response.append(f"🗺️ **LỘ TRÌNH NHIỀU CHẶNG** (Khoảng {int(total_time)} phút)")
                response.append("="*30)

                current_rid = route_ids[0]
                start_seg = stop_names[0]
                step = 1

                for i in range(len(route_ids)):
                    if route_ids[i] != current_rid:
                        # Đổi xe
                        bus_no = route_map.get(current_rid, "?")
                        response.append(f"{step}. Đi xe **{bus_no}** từ '{start_seg}' đến '{stop_names[i]}'.")
                        response.append(f"   ⬇ (Xuống trạm, đổi xe)")
                        current_rid = route_ids[i]
                        start_seg = stop_names[i]
                        step += 1

                # Chặng cuối
                bus_no = route_map.get(current_rid, "?")
                response.append(f"{step}. Đi xe **{bus_no}** từ '{start_seg}' đến đích '{stop_names[-1]}'.")
                
                return "\n".join(response)

            return "❌ Hai địa điểm này quá xa."

    def chat(self, text):
        if "đến" in text.lower():
            try:
                parts = text.lower().split("đến")
                start = parts[0].replace("đi từ", "").replace("tìm đường", "").strip()
                end = parts[1].strip()
                return self.solve_route(start, end)
            except: pass
        return "🤖 Bot: Hãy nhập 'Đi từ [A] đến [B]'"

if __name__ == "__main__":
    bot = BusBotFinal()
    print("\n👋 BUS MAP FULL OPTIONS (V6.1)")
    while True:
        q = input("\n👤 Bạn: ")
        if q.lower() in ['exit', 'quit']: break
        print(bot.chat(q))


👋 BUS MAP FULL OPTIONS (V6.1)
      🔎 Đang tìm trạm cho: 'chợ bến xe buýt sài gòn'...
      🌍 Không thấy tên trạm. Đang tìm địa điểm trên bản đồ số...
      🔎 Đang tìm trạm cho: 'thảo cầm viên'...
❌ Bó tay. Không tìm thấy địa điểm 'chợ bến thành'.
      🔎 Đang tìm trạm cho: 'thảo cầm viên'...
      🔎 Đang tìm trạm cho: 'đại học tôn đức thắng'...
      ✅ Chốt lộ trình: Thảo Cầm Viên -> Đại học Tôn Đức Thắng

🔄 **LỘ TRÌNH 1 LẦN ĐỔI XE**
   1️⃣ Đi xe **52** từ 'Thảo Cầm Viên' đến trạm 'Tòa án Nhân dân Thành Phố'.
   ⬇ (Xuống xe, chờ xíu)
   2️⃣ Đón tiếp xe **31** đi tới 'Đại học Tôn Đức Thắng'.
      🔎 Đang tìm trạm cho: 'đại học đại học tôn đức thắng'...
      🔎 Đang tìm trạm cho: 'đại học trường đại học văn lang'...
      ✅ Chốt lộ trình: Đại học Tôn Đức Thắng -> Trường Đại học Văn Lang

🔄 **LỘ TRÌNH 1 LẦN ĐỔI XE**
   1️⃣ Đi xe **44** từ 'Đại học Tôn Đức Thắng' đến trạm 'Chùa Anh Linh'.
   ⬇ (Xuống xe, chờ xíu)
   2️⃣ Đón tiếp xe **31** đi tới 'Trường Đại học Văn Lang'.
      🔎 Đang tì

### Test V6 (Bản này oke)

In [3]:
import time
import re
import requests
from neo4j import GraphDatabase

# --- CẤU HÌNH ---
URI = "neo4j+s://403c0411.databases.neo4j.io"
AUTH = ("neo4j", "20qU7hUdIdlzyXii3tmnCDgvAFkUh-NobhE52Oq7Dvw")

class BusBotV12:
    def __init__(self):
        self.driver = GraphDatabase.driver(URI, auth=AUTH)
        
        # --- BỘ TỪ ĐIỂN KHỔNG LỒ ---
        self.ABBREVIATIONS = {
            # 1. TRƯỜNG ĐẠI HỌC
            "tdt": "đại học tôn đức thắng", "đh tdt": "đại học tôn đức thắng", "tôn đức thắng": "đại học tôn đức thắng",
            "văn lang": "trường đại học văn lang", "đh văn lang": "trường đại học văn lang", "vlu": "trường đại học văn lang",
            "csnd": "đại học cảnh sát nhân dân", "đh csnd": "đại học cảnh sát nhân dân",
            "bk": "đại học bách khoa", "bách khoa": "đại học bách khoa", "hcmut": "đại học bách khoa",
            "khtn": "đại học khoa học tự nhiên", "tự nhiên": "đại học khoa học tự nhiên", "hcmus": "đại học khoa học tự nhiên",
            "ussh": "đại học khoa học xã hội và nhân văn", "nhân văn": "đại học khoa học xã hội và nhân văn",
            "spkt": "đại học sư phạm kỹ thuật", "sư phạm kỹ thuật": "đại học sư phạm kỹ thuật", "ute": "đại học sư phạm kỹ thuật",
            "nlu": "đại học nông lâm", "nông lâm": "đại học nông lâm",
            "ueh": "đại học kinh tế", "kinh tế": "đại học kinh tế",
            "ulu": "đại học luật", "luật": "đại học luật",
            "yds": "đại học y dược", "y dược": "đại học y dược",
            "hutech": "đại học công nghệ tphcm", "công nghệ": "đại học công nghệ tphcm",
            "uef": "đại học kinh tế tài chính",
            "rmit": "đại học rmit",
            "fpt": "đại học fpt",
            "hoa sen": "đại học hoa sen", "hsu": "đại học hoa sen",
            "sgu": "đại học sài gòn", "sài gòn": "đại học sài gòn",
            "iuh": "đại học công nghiệp", "công nghiệp": "đại học công nghiệp",
            "ntt": "đại học nguyễn tất thành", "nguyễn tất thành": "đại học nguyễn tất thành",
            "ou": "đại học mở", "mở": "đại học mở",
            
            # 2. BẾN XE & ĐẦU MỐI
            "bến thành": "bến xe buýt sài gòn", "chợ bến thành": "bến xe buýt sài gòn", "bx sài gòn": "bến xe buýt sài gòn",
            "chợ lớn": "bến xe buýt chợ lớn", "bx chợ lớn": "bến xe buýt chợ lớn",
            "miền đông": "bến xe miền đông", "bx miền đông": "bến xe miền đông",
            "miền tây": "bến xe miền tây", "bx miền tây": "bến xe miền tây",
            "an sương": "bến xe an sương", "bx an sương": "bến xe an sương",
            "ngã tư ga": "bến xe ngã tư ga", "bx ngã tư ga": "bến xe ngã tư ga",
            "củ chi": "bến xe củ chi", "bx củ chi": "bến xe củ chi",
            "quận 8": "bến xe quận 8", "bx quận 8": "bến xe quận 8",
            
            # 3. ĐỊA DANH / VUI CHƠI
            "suối tiên": "khu du lịch suối tiên", "kdl suối tiên": "khu du lịch suối tiên",
            "đầm sen": "công viên văn hóa đầm sen", "cv đầm sen": "công viên văn hóa đầm sen",
            "thảo cầm viên": "thảo cầm viên", "sở thú": "thảo cầm viên",
            "nhà thờ đức bà": "công xã paris",
            "dinh độc lập": "dinh độc lập",
            "phố đi bộ": "nguyễn huệ",
            "bitexco": "hồ tùng mậu",
            "landmark 81": "tân cảng", "vinhoems": "tân cảng",
            "sân bay": "sân bay tân sơn nhất", "tsn": "sân bay tân sơn nhất",
            "aeon": "aeon mall", "aeon tân phú": "aeon mall tân phú",
            "gigamall": "gigamall",
            "chợ rẫy": "bệnh viện chợ rẫy", "bv chợ rẫy": "bệnh viện chợ rẫy",
            "115": "bệnh viện nhân dân 115",
            
            # 4. CHUNG
            "đh": "đại học", "trường đh": "đại học",
            "cđ": "cao đẳng", "trường cđ": "cao đẳng",
            "bv": "bệnh viện", "bệnh viện": "bệnh viện",
            "kdc": "khu dân cư", "cc": "chung cư", 
            "ubnd": "ủy ban nhân dân", "q.": "quận ", "p.": "phường ",
        }
        
        # --- LOGIC REGEX 1 LƯỢT ---
        sorted_keys = sorted(self.ABBREVIATIONS.keys(), key=len, reverse=True)
        self.pattern = re.compile(r'\b(' + '|'.join(map(re.escape, sorted_keys)) + r')\b')

    def close(self):
        self.driver.close()

    def normalize_query(self, text):
        text = text.lower()
        def replace(match):
            return self.ABBREVIATIONS[match.group(0)]
        return self.pattern.sub(replace, text)

    # --- GEOLOCATION ---
    def get_coordinates(self, place_name):
        try:
            url = "https://nominatim.openstreetmap.org/search"
            params = {'q': f"{place_name}, Ho Chi Minh City, Vietnam", 'format': 'json', 'limit': 1}
            headers = {'User-Agent': 'BusMapBot/1.0'}
            resp = requests.get(url, params=params, headers=headers, timeout=2)
            data = resp.json()
            if data: return float(data[0]['lat']), float(data[0]['lon']), data[0]['display_name']
        except: pass
        return None, None, None

    def find_nearest_station_by_coords(self, session, lat, lng):
        query = """
        MATCH (n:BusStop)
        WITH n, point.distance(point({latitude: n.lat, longitude: n.lng}), point({latitude: $lat, longitude: $lng})) AS dist
        WHERE dist < 800 RETURN n ORDER BY dist ASC LIMIT 1
        """
        result = session.run(query, lat=lat, lng=lng).single()
        if result: return result['n']
        return None

    # --- TÌM TRẠM (MULTI-CANDIDATE) ---
    def find_stop_candidates(self, session, query_text):
        clean_query = self.normalize_query(query_text).replace("trạm", "").strip()
        clean_query = clean_query.replace("trường đại học trường đại học", "trường đại học")
        
        print(f"      🔎 Đang tìm: '{clean_query}'...")

        q_exact = """
        MATCH (n:BusStop) 
        WHERE toLower(n.name) CONTAINS $txt 
           OR toLower(n.search) CONTAINS $txt
           OR toLower(n.code) = $txt
        RETURN n LIMIT 5
        """
        # --- FIX LỖI Ở ĐÂY: Dùng list() bao ngoài session.run() ---
        res = list(session.run(q_exact, txt=clean_query)) 
        candidates = [r['n'] for r in res]
        
        if not candidates:
            print(f"      🌍 Tìm trên bản đồ số...")
            lat, lng, addr = self.get_coordinates(clean_query)
            if lat and lng:
                print(f"         📍 Địa điểm: {addr[:30]}...")
                nearest = self.find_nearest_station_by_coords(session, lat, lng)
                if nearest: candidates.append(nearest)

        return candidates 

    def solve_route(self, start_text, end_text):
        with self.driver.session() as session:
            s_candidates = self.find_stop_candidates(session, start_text)
            e_candidates = self.find_stop_candidates(session, end_text)

            if not s_candidates: return f"❌ Không tìm thấy: '{start_text}'"
            if not e_candidates: return f"❌ Không tìm thấy: '{end_text}'"

            print(f"      ⚡ Đang quét {len(s_candidates)}x{len(e_candidates)} cặp trạm...")
            
            # ƯU TIÊN 1: ĐI THẲNG
            for s in s_candidates:
                for e in e_candidates:
                    q_direct = """
                    MATCH (s:BusStop {id: $sid})-[r1:ON_ROUTE]->(route:BusRoute)<-[r2:ON_ROUTE]-(e:BusStop {id: $eid})
                    RETURN route.route_no, route.name LIMIT 1
                    """
                    direct = session.run(q_direct, sid=s['id'], eid=e['id']).single()
                    if direct:
                        return (f"\n🎯 **TÌM THẤY XE ĐI THẲNG!**\n"
                                f"   🚌 **Đi xe {direct['route.route_no']}**: {direct['route.name']}\n"
                                f"   📍 Từ: {s['name']}\n   🏁 Đến: {e['name']}")

            # ƯU TIÊN 2: TRUNG CHUYỂN
            s_best, e_best = s_candidates[0], e_candidates[0]
            q_transfer = """
            MATCH (s:BusStop {id: $sid})-[r1:ON_ROUTE]->(rA:BusRoute)<-[r2:ON_ROUTE]-(mid:BusStop)
            MATCH (mid)-[r3:ON_ROUTE]->(rB:BusRoute)<-[r4:ON_ROUTE]-(e:BusStop {id: $eid})
            WHERE rA <> rB
            RETURN rA.route_no, rB.route_no, mid.name LIMIT 1
            """
            one_stop = session.run(q_transfer, sid=s_best['id'], eid=e_best['id']).single()
            if one_stop:
                return (f"\n🔄 **LỘ TRÌNH 1 LẦN ĐỔI XE**\n"
                        f"   1️⃣ Đi xe **{one_stop['rA.route_no']}** từ '{s_best['name']}' đến '{one_stop['mid.name']}'.\n"
                        f"   2️⃣ Đổi sang xe **{one_stop['rB.route_no']}** đi tiếp.")

            # ƯU TIÊN 3: GRAPH
            q_graph = """
            MATCH (start:BusStop {id: $sid}), (end:BusStop {id: $eid})
            MATCH p = shortestPath((start)-[:NEXT_HOP*..100]->(end)) 
            WITH p, relationships(p) AS rels, nodes(p) AS stops
            RETURN [r IN rels | r.route_id] AS route_ids, [s IN stops | s.name] AS stop_names, reduce(t=0, r IN rels | t + r.time) AS total_time
            """
            result = session.run(q_graph, sid=s_best['id'], eid=e_best['id']).single()

            if result:
                route_ids = result['route_ids']
                stop_names = result['stop_names']
                total_time = result['total_time']
                unique_rids = list(set(route_ids))
                route_map = {}
                if unique_rids:
                    # --- FIX LỖI Ở ĐÂY LUÔN ---
                    r_info = list(session.run("MATCH (r:BusRoute) WHERE r.id IN $rids RETURN r.id, r.route_no", rids=unique_rids))
                    for r in r_info: route_map[r['r.id']] = r['r.route_no']

                response = [f"\n🗺️ **LỘ TRÌNH KẾT HỢP** (Khoảng {int(total_time)} phút)"]
                current_rid = route_ids[0]
                start_seg = stop_names[0]
                step = 1
                for i in range(len(route_ids)):
                    if route_ids[i] != current_rid:
                        bus_no = route_map.get(current_rid, "?")
                        response.append(f"{step}. Đi xe **{bus_no}** từ '{start_seg}' đến '{stop_names[i]}'.")
                        response.append(f"   ⬇ (Xuống trạm, đổi xe)")
                        current_rid = route_ids[i]
                        start_seg = stop_names[i]
                        step += 1
                bus_no = route_map.get(current_rid, "?")
                response.append(f"{step}. Đi xe **{bus_no}** từ '{start_seg}' đến đích '{stop_names[-1]}'.")
                return "\n".join(response)

            return "❌ Quá xa hoặc không có đường đi."

    def chat(self, text):
        if "đến" in text.lower():
            try:
                parts = text.lower().split("đến")
                start = parts[0].replace("đi từ", "").replace("tìm đường", "").strip()
                end = parts[1].strip()
                if not start or not end: return "⚠️ Vui lòng nhập đủ điểm đi và đến."
                return self.solve_route(start, end)
            except Exception as e:
                # In chi tiết lỗi để debug nếu có sự cố
                import traceback
                traceback.print_exc()
                return f"⚠️ Lỗi hệ thống: {e}"
        return "🤖 Bot: Hãy nhập 'Đi từ [A] đến [B]'"

if __name__ == "__main__":
    bot = BusBotV12()
    print("\n👋 BUS MAP V12: BẢN FIX LỖI DRIVER (ĐÃ TEST)")
    while True:
        q = input("\n👤 Bạn: ")
        if q.lower() in ['exit', 'quit']: break
        print(bot.chat(q))


👋 BUS MAP V12: BẢN FIX LỖI DRIVER (ĐÃ TEST)
      🔎 Đang tìm: 'đại học tôn đức thắng'...
      🔎 Đang tìm: 'trường đại học văn lang'...
      ⚡ Đang quét 3x1 cặp trạm...

🎯 **TÌM THẤY XE ĐI THẲNG!**
   🚌 **Đi xe 31**: Bus 31: Đại Học Tôn Đức Thắng - Bến Thành - Đại học Văn Lang
   📍 Từ: Đại học Tôn Đức Thắng
   🏁 Đến: Trường Đại học Văn Lang
      🔎 Đang tìm: 'trường đại học văn lang'...
      🔎 Đang tìm: 'đại học đại học tôn đức thắng'...
      🌍 Tìm trên bản đồ số...
         📍 Địa điểm: Tượng đài Tôn Đức Thắng, Đường...
      ⚡ Đang quét 1x1 cặp trạm...

🔄 **LỘ TRÌNH 1 LẦN ĐỔI XE**
   1️⃣ Đi xe **31** từ 'Trường Đại học Văn Lang' đến 'Nguyễn Kim'.
   2️⃣ Đổi sang xe **86** đi tiếp.
      🔎 Đang tìm: 'bến xe buýt sài gòn'...
      🔎 Đang tìm: 'thảo cầm viên'...
      ⚡ Đang quét 1x2 cặp trạm...

🎯 **TÌM THẤY XE ĐI THẲNG!**
   🚌 **Đi xe 19**: Bus 19: Bến Thành - Khu chế xuất Linh Trung - Đại học Quốc gia
   📍 Từ: Bến xe buýt Sài Gòn
   🏁 Đến: Thảo Cầm Viên
      🔎 Đang tìm: 'công xã 

### Test V7 (Xài được ròi)

In [ ]:
import time
import re
import requests
from neo4j import GraphDatabase

# --- CẤU HÌNH ---
URI = "neo4j+s://403c0411.databases.neo4j.io"
AUTH = ("neo4j", "20qU7hUdIdlzyXii3tmnCDgvAFkUh-NobhE52Oq7Dvw")

class BusBotV13:
    def __init__(self):
        self.driver = GraphDatabase.driver(URI, auth=AUTH)
        
        self.ABBREVIATIONS = {
            # 1. TRƯỜNG ĐẠI HỌC (Full option)
            "tdt": "đại học tôn đức thắng", "đh tdt": "đại học tôn đức thắng",
            "văn lang": "trường đại học văn lang", "đh văn lang": "trường đại học văn lang", "vlu": "trường đại học văn lang",
            "csnd": "đại học cảnh sát nhân dân", "đh csnd": "đại học cảnh sát nhân dân",
            "bk": "đại học bách khoa", "bách khoa": "đại học bách khoa",
            "khtn": "đại học khoa học tự nhiên", "tự nhiên": "đại học khoa học tự nhiên",
            "ussh": "đại học khoa học xã hội và nhân văn", "nhân văn": "đại học khoa học xã hội và nhân văn",
            "spkt": "đại học sư phạm kỹ thuật", "sư phạm kỹ thuật": "đại học sư phạm kỹ thuật",
            "nlu": "đại học nông lâm", "nông lâm": "đại học nông lâm",
            "ueh": "đại học kinh tế", "kinh tế": "đại học kinh tế",
            "ulu": "đại học luật", "luật": "đại học luật",
            "yds": "đại học y dược", "y dược": "đại học y dược",
            "hutech": "đại học công nghệ tphcm", "công nghệ": "đại học công nghệ tphcm",
            "uef": "đại học kinh tế tài chính",
            "rmit": "đại học rmit", "fpt": "đại học fpt",
            "hoa sen": "đại học hoa sen", "hsu": "đại học hoa sen",
            "sgu": "đại học sài gòn", "sài gòn": "đại học sài gòn",
            "iuh": "đại học công nghiệp", "công nghiệp": "đại học công nghiệp",
            "ntt": "đại học nguyễn tất thành", "nguyễn tất thành": "đại học nguyễn tất thành",
            "ou": "đại học mở", "mở": "đại học mở",
            
            # 2. ĐỊA ĐIỂM HOT
            "bến thành": "bến xe buýt sài gòn", "chợ bến thành": "bến xe buýt sài gòn",
            "suối tiên": "khu du lịch suối tiên", "kdl suối tiên": "khu du lịch suối tiên",
            "đầm sen": "công viên văn hóa đầm sen", "cv đầm sen": "công viên văn hóa đầm sen","công viên nước đầm sen": "công viên văn hóa đầm sen",
            "thảo cầm viên": "thảo cầm viên",
            "nhà thờ đức bà": "công xã paris",
            "sân bay": "sân bay tân sơn nhất", "tsn": "sân bay tân sơn nhất",
            "chợ rẫy": "bệnh viện chợ rẫy", "115": "bệnh viện nhân dân 115",
            "lotte mart nam sài gòn":"Lotte Mart","lotte mart quận 7":"Lotte Mart",
            
            # 3. TỪ VIẾT TẮT CHUNG
            "đh": "đại học", "trường đh": "đại học",
            "cđ": "cao đẳng", "bv": "bệnh viện",
            "kdc": "khu dân cư", "bx": "bến xe", "q.": "quận ",
        }
        
        # Regex 1 lượt
        sorted_keys = sorted(self.ABBREVIATIONS.keys(), key=len, reverse=True)
        self.pattern = re.compile(r'\b(' + '|'.join(map(re.escape, sorted_keys)) + r')\b')

    def close(self):
        self.driver.close()

    def normalize_query(self, text):
        text = text.lower()
        def replace(match):
            return self.ABBREVIATIONS[match.group(0)]
        text = self.pattern.sub(replace, text)
        
        # --- FIX LỖI LẶP TỪ (MỚI) ---
        # Xử lý các trường hợp như "đại học đại học", "bến xe bến xe"
        text = text.replace("đại học đại học", "đại học")
        text = text.replace("trường đại học trường đại học", "trường đại học")
        text = text.replace("bến xe bến xe", "bến xe")
        text = text.replace("bệnh viện bệnh viện", "bệnh viện")
        return text

    # --- GEOLOCATION ---
    def get_coordinates(self, place_name):
        try:
            url = "https://nominatim.openstreetmap.org/search"
            params = {'q': f"{place_name}, Ho Chi Minh City, Vietnam", 'format': 'json', 'limit': 1}
            headers = {'User-Agent': 'BusMapBot/1.0'}
            resp = requests.get(url, params=params, headers=headers, timeout=2)
            data = resp.json()
            if data: return float(data[0]['lat']), float(data[0]['lon']), data[0]['display_name']
        except: pass
        return None, None, None

    def find_nearest_station_by_coords(self, session, lat, lng):
        query = """
        MATCH (n:BusStop)
        WITH n, point.distance(point({latitude: n.lat, longitude: n.lng}), point({latitude: $lat, longitude: $lng})) AS dist
        WHERE dist < 800 RETURN n ORDER BY dist ASC LIMIT 1
        """
        result = session.run(query, lat=lat, lng=lng).single()
        if result: return result['n']
        return None

    # --- TÌM TRẠM ---
    def find_stop_candidates(self, session, query_text):
        clean_query = self.normalize_query(query_text).replace("trạm", "").strip()
        print(f"      🔎 Đang tìm: '{clean_query}'...")

        q_exact = """
        MATCH (n:BusStop) 
        WHERE toLower(n.name) CONTAINS $txt 
           OR toLower(n.search) CONTAINS $txt
           OR toLower(n.code) = $txt
        RETURN n LIMIT 5
        """
        # Fix lỗi Driver .list()
        res = list(session.run(q_exact, txt=clean_query))
        candidates = [r['n'] for r in res]
        
        if not candidates:
            print(f"      🌍 Tìm trên bản đồ số...")
            lat, lng, addr = self.get_coordinates(clean_query)
            if lat and lng:
                print(f"         📍 Địa điểm: {addr[:30]}...")
                nearest = self.find_nearest_station_by_coords(session, lat, lng)
                if nearest: candidates.append(nearest)

        return candidates 

    def solve_route(self, start_text, end_text):
        with self.driver.session() as session:
            s_candidates = self.find_stop_candidates(session, start_text)
            e_candidates = self.find_stop_candidates(session, end_text)

            if not s_candidates: return f"❌ Không tìm thấy: '{start_text}'"
            if not e_candidates: return f"❌ Không tìm thấy: '{end_text}'"

            print(f"      ⚡ Đang quét {len(s_candidates)}x{len(e_candidates)} cặp trạm...")
            
            # ƯU TIÊN 1: ĐI THẲNG
            for s in s_candidates:
                for e in e_candidates:
                    q_direct = """
                    MATCH (s:BusStop {id: $sid})-[r1:ON_ROUTE]->(route:BusRoute)<-[r2:ON_ROUTE]-(e:BusStop {id: $eid})
                    RETURN route.route_no, route.name LIMIT 1
                    """
                    direct = session.run(q_direct, sid=s['id'], eid=e['id']).single()
                    if direct:
                        return (f"\n🎯 **TÌM THẤY XE ĐI THẲNG!**\n"
                                f"   🚌 **Đi xe {direct['route.route_no']}**: {direct['route.name']}\n"
                                f"   📍 Từ: {s['name']}\n   🏁 Đến: {e['name']}")

            # ƯU TIÊN 2: TRUNG CHUYỂN
            s_best, e_best = s_candidates[0], e_candidates[0]
            q_transfer = """
            MATCH (s:BusStop {id: $sid})-[r1:ON_ROUTE]->(rA:BusRoute)<-[r2:ON_ROUTE]-(mid:BusStop)
            MATCH (mid)-[r3:ON_ROUTE]->(rB:BusRoute)<-[r4:ON_ROUTE]-(e:BusStop {id: $eid})
            WHERE rA <> rB
            RETURN rA.route_no, rB.route_no, mid.name LIMIT 1
            """
            one_stop = session.run(q_transfer, sid=s_best['id'], eid=e_best['id']).single()
            if one_stop:
                return (f"\n🔄 **LỘ TRÌNH 1 LẦN ĐỔI XE**\n"
                        f"   1️⃣ Đi xe **{one_stop['rA.route_no']}** từ '{s_best['name']}' đến '{one_stop['mid.name']}'.\n"
                        f"   2️⃣ Đổi sang xe **{one_stop['rB.route_no']}** đi tiếp.")

            # ƯU TIÊN 3: GRAPH
            q_graph = """
            MATCH (start:BusStop {id: $sid}), (end:BusStop {id: $eid})
            MATCH p = shortestPath((start)-[:NEXT_HOP*..100]->(end)) 
            WITH p, relationships(p) AS rels, nodes(p) AS stops
            RETURN [r IN rels | r.route_id] AS route_ids, [s IN stops | s.name] AS stop_names, reduce(t=0, r IN rels | t + r.time) AS total_time
            """
            result = session.run(q_graph, sid=s_best['id'], eid=e_best['id']).single()

            if result:
                route_ids = result['route_ids']
                stop_names = result['stop_names']
                total_time = result['total_time']
                unique_rids = list(set(route_ids))
                route_map = {}
                if unique_rids:
                    r_info = list(session.run("MATCH (r:BusRoute) WHERE r.id IN $rids RETURN r.id, r.route_no", rids=unique_rids))
                    for r in r_info: route_map[r['r.id']] = r['r.route_no']

                response = [f"\n🗺️ **LỘ TRÌNH KẾT HỢP** (Khoảng {int(total_time)} phút)"]
                current_rid = route_ids[0]
                start_seg = stop_names[0]
                step = 1
                for i in range(len(route_ids)):
                    if route_ids[i] != current_rid:
                        bus_no = route_map.get(current_rid, "?")
                        response.append(f"{step}. Đi xe **{bus_no}** từ '{start_seg}' đến '{stop_names[i]}'.")
                        response.append(f"   ⬇ (Xuống trạm, đổi xe)")
                        current_rid = route_ids[i]
                        start_seg = stop_names[i]
                        step += 1
                bus_no = route_map.get(current_rid, "?")
                response.append(f"{step}. Đi xe **{bus_no}** từ '{start_seg}' đến đích '{stop_names[-1]}'.")
                return "\n".join(response)

            return "❌ Quá xa hoặc không có đường đi."

    def chat(self, text):
        if "đến" in text.lower():
            try:
                parts = text.lower().split("đến")
                start = parts[0].replace("đi từ", "").replace("tìm đường", "").strip()
                end = parts[1].strip()
                if not start or not end: return "⚠️ Vui lòng nhập đủ điểm đi và đến."
                return self.solve_route(start, end)
            except Exception as e:
                return f"⚠️ Lỗi hệ thống: {e}"
        return "🤖 Bot: Hãy nhập 'Đi từ [A] đến [B]'"

if __name__ == "__main__":
    bot = BusBotV13()
    print("\n👋 Tìm đường đi bằng xe bus!")
    while True:
        q = input("\n👤 Bạn: ")
        if q.lower() in ['exit', 'quit']: break
        print(bot.chat(q))


👋 Tìm đường đi bằng xe bus!
      🔎 Đang tìm: 'công viên văn hóa đầm sen'...
      🌍 Tìm trên bản đồ số...
         📍 Địa điểm: Công viên Văn hóa Đầm Sen, Hẻm...
      🔎 Đang tìm: 'đại học tôn đức thắng'...
      ⚡ Đang quét 1x3 cặp trạm...

🗺️ **LỘ TRÌNH KẾT HỢP** (Khoảng 26 phút)
1. Đi xe **145** từ 'Cổng 1 Đầm Sen' đến 'Trường Nguyễn Đức Cảnh'.
   ⬇ (Xuống trạm, đổi xe)
2. Đi xe **09** từ 'Trường Nguyễn Đức Cảnh' đến 'Chùa Nam Phổ Đà'.
   ⬇ (Xuống trạm, đổi xe)
3. Đi xe **62-7** từ 'Chùa Nam Phổ Đà' đến 'Bến xe Chợ Lớn'.
   ⬇ (Xuống trạm, đổi xe)
4. Đi xe **06** từ 'Bến xe Chợ Lớn' đến 'Bệnh viện Chợ Rẫy'.
   ⬇ (Xuống trạm, đổi xe)
5. Đi xe **150** từ 'Bệnh viện Chợ Rẫy' đến 'Đại học Y Dược'.
   ⬇ (Xuống trạm, đổi xe)
6. Đi xe **06** từ 'Đại học Y Dược' đến 'Bệnh viện Đại học Y Dược'.
   ⬇ (Xuống trạm, đổi xe)
7. Đi xe **07** từ 'Bệnh viện Đại học Y Dược' đến 'Trường Trần Khai Nguyên'.
   ⬇ (Xuống trạm, đổi xe)
8. Đi xe **150** từ 'Trường Trần Khai Nguyên' đến 'Trường Khiếm thị Ngu

Vấn đề nằm ở chỗ bạn dùng thuật toán Shortest Path (Đường đi ngắn nhất) lên nhầm loại cạnh (Relationship).

Cấu trúc dữ liệu: Trong Neo4j của bạn có 2 loại quan hệ chính:

[:NEXT_HOP]: Nối trạm A -> trạm B (Trạm kế tiếp trên đường đi).

[:ON_ROUTE]: Nối Trạm -> Tuyến Bus (Trạm này thuộc tuyến nào).

Code cũ: Bạn dùng shortestPath((start)-[:NEXT_HOP*]-(end)).

Tư duy của máy: "Tôi cần tìm đường đi qua ít node nhất".

Thực tế: Để đi từ Quận 1 sang Quận 7, nếu đi 1 tuyến xe buýt (ví dụ số 34), bạn phải ngồi qua 30 trạm dừng (tức là đi qua 30 node).

Đường tắt tai hại: Máy tính phát hiện ra nếu nhảy cóc: Đi xe A 2 trạm, xuống bắt xe B đi 3 trạm, rồi bắt xe C... tổng cộng chỉ đi qua 15 node thôi.

Kết quả: Máy thấy 15 < 30, nên nó chọn lộ trình "nhảy cóc" 16 lần đó. Nó không hiểu rằng việc "xuống xe chờ chuyến khác" là cực hình với con người.

### Tìm tuyến đường quá dài nên tối ưu, tối đa 5 lần trung chuyển

In [3]:
import time
import re
import requests
from neo4j import GraphDatabase

# --- CẤU HÌNH ---
URI = "neo4j+s://403c0411.databases.neo4j.io"
AUTH = ("neo4j", "20qU7hUdIdlzyXii3tmnCDgvAFkUh-NobhE52Oq7Dvw")

class BusBotV13:
    def __init__(self):
        self.driver = GraphDatabase.driver(URI, auth=AUTH)
        
        self.ABBREVIATIONS = {
            # 1. TRƯỜNG ĐẠI HỌC
            "tdt": "đại học tôn đức thắng", "đh tdt": "đại học tôn đức thắng",
            "văn lang": "trường đại học văn lang", "đh văn lang": "trường đại học văn lang", "vlu": "trường đại học văn lang",
            "csnd": "đại học cảnh sát nhân dân", "đh csnd": "đại học cảnh sát nhân dân",
            "bk": "đại học bách khoa", "bách khoa": "đại học bách khoa",
            "khtn": "đại học khoa học tự nhiên", "tự nhiên": "đại học khoa học tự nhiên",
            "ussh": "đại học khoa học xã hội và nhân văn", "nhân văn": "đại học khoa học xã hội và nhân văn",
            "spkt": "đại học sư phạm kỹ thuật", "sư phạm kỹ thuật": "đại học sư phạm kỹ thuật",
            "nlu": "đại học nông lâm", "nông lâm": "đại học nông lâm",
            "ueh": "đại học kinh tế", "kinh tế": "đại học kinh tế",
            "ulu": "đại học luật", "luật": "đại học luật",
            "yds": "đại học y dược", "y dược": "đại học y dược",
            "hutech": "đại học công nghệ tphcm", "công nghệ": "đại học công nghệ tphcm",
            "uef": "đại học kinh tế tài chính",
            "rmit": "đại học rmit", "fpt": "đại học fpt",
            "hoa sen": "đại học hoa sen", "hsu": "đại học hoa sen",
            "sgu": "đại học sài gòn", "sài gòn": "đại học sài gòn",
            "iuh": "đại học công nghiệp", "công nghiệp": "đại học công nghiệp",
            "ntt": "đại học nguyễn tất thành", "nguyễn tất thành": "đại học nguyễn tất thành",
            "ou": "đại học mở", "mở": "đại học mở",
            
            # 2. ĐỊA ĐIỂM HOT
            "bến thành": "bến xe buýt sài gòn", "chợ bến thành": "bến xe buýt sài gòn",
            "suối tiên": "khu du lịch suối tiên", "kdl suối tiên": "khu du lịch suối tiên",
            "đầm sen": "công viên văn hóa đầm sen", "cv đầm sen": "công viên văn hóa đầm sen", "công viên nước đầm sen": "công viên văn hóa đầm sen",
            "thảo cầm viên": "thảo cầm viên",
            "nhà thờ đức bà": "công xã paris",
            "sân bay": "sân bay tân sơn nhất", "tsn": "sân bay tân sơn nhất","sân bay tsn": "sân bay tân sơn nhất",
            "chợ rẫy": "bệnh viện chợ rẫy", "115": "bệnh viện nhân dân 115",
            "lotte mart nam sài gòn":"Lotte Mart","lotte mart quận 7":"Lotte Mart",
            
            # 3. TỪ VIẾT TẮT CHUNG
            "đh": "đại học", "trường đh": "đại học",
            "cđ": "cao đẳng", "bv": "bệnh viện",
            "kdc": "khu dân cư", "bx": "bến xe", "q.": "quận ",
        }
        
        sorted_keys = sorted(self.ABBREVIATIONS.keys(), key=len, reverse=True)
        self.pattern = re.compile(r'\b(' + '|'.join(map(re.escape, sorted_keys)) + r')\b')

    def close(self):
        self.driver.close()

    def normalize_query(self, text):
        text = text.lower()
        def replace(match):
            return self.ABBREVIATIONS[match.group(0)]
        text = self.pattern.sub(replace, text)
        
        # Fix lỗi lặp từ
        text = text.replace("đại học đại học", "đại học")
        text = text.replace("trường đại học trường đại học", "trường đại học")
        text = text.replace("bến xe bến xe", "bến xe")
        text = text.replace("bệnh viện bệnh viện", "bệnh viện")
        text = text.replace("công viên văn hóa công viên văn hóa", "công viên văn hóa")
        return text

    # --- GEOLOCATION ---
    def get_coordinates(self, place_name):
        try:
            url = "https://nominatim.openstreetmap.org/search"
            # Thêm context Ho Chi Minh City để tìm chính xác hơn
            params = {'q': f"{place_name}, Ho Chi Minh City, Vietnam", 'format': 'json', 'limit': 1}
            headers = {'User-Agent': 'BusMapBot/1.0'}
            resp = requests.get(url, params=params, headers=headers, timeout=2)
            data = resp.json()
            if data: return float(data[0]['lat']), float(data[0]['lon']), data[0]['display_name']
        except: pass
        return None, None, None

    def find_nearest_station_by_coords(self, session, lat, lng):
        query = """
        MATCH (n:BusStop)
        WITH n, point.distance(point({latitude: n.lat, longitude: n.lng}), point({latitude: $lat, longitude: $lng})) AS dist
        WHERE dist < 1000 
        RETURN n ORDER BY dist ASC LIMIT 1
        """
        result = session.run(query, lat=lat, lng=lng).single()
        if result: return result['n']
        return None

    # --- TÌM TRẠM (LOGIC MỚI: FALLBACK) ---
    def _internal_search(self, session, txt):
        """Hàm phụ tìm trong DB hoặc Map"""
        q_exact = """
        MATCH (n:BusStop) 
        WHERE toLower(n.name) CONTAINS $txt 
           OR toLower(n.search) CONTAINS $txt
           OR toLower(n.code) = $txt
        RETURN n LIMIT 5
        """
        res = list(session.run(q_exact, txt=txt))
        candidates = [r['n'] for r in res]
        
        if not candidates:
            lat, lng, addr = self.get_coordinates(txt)
            if lat and lng:
                print(f"      🌍 Map tìm thấy: {addr[:30]}...")
                nearest = self.find_nearest_station_by_coords(session, lat, lng)
                if nearest: candidates.append(nearest)
        return candidates

    def find_stop_candidates(self, session, query_text):
        # 1. Tìm bằng từ khóa đã chuẩn hóa
        clean_query = self.normalize_query(query_text).replace("trạm", "").strip()
        print(f"      🔎 Đang tìm (Normalized): '{clean_query}'...")
        
        candidates = self._internal_search(session, clean_query)

        # 2. Nếu không thấy -> Tìm bằng từ gốc (Fix lỗi công viên nước đầm sen)
        if not candidates and clean_query != query_text.strip():
            raw_query = query_text.replace("trạm", "").strip()
            print(f"      ⚠️ Không thấy -> Thử tìm từ gốc: '{raw_query}'...")
            candidates = self._internal_search(session, raw_query)

        return candidates 

    # --- TÌM ĐƯỜNG (LOGIC MỚI: TUYẾN XE) ---
    def solve_route(self, start_text, end_text):
        with self.driver.session() as session:
            s_candidates = self.find_stop_candidates(session, start_text)
            e_candidates = self.find_stop_candidates(session, end_text)

            if not s_candidates: return f"❌ Không tìm thấy điểm đi: '{start_text}'"
            if not e_candidates: return f"❌ Không tìm thấy điểm đến: '{end_text}'"

            # Lấy danh sách ID để query cho nhanh
            s_ids = [s['id'] for s in s_candidates]
            e_ids = [e['id'] for e in e_candidates]
            
            print(f"      ⚡ Đang quét lộ trình từ {len(s_ids)} trạm đi đến {len(e_ids)} trạm đích...")
            
            # ƯU TIÊN 1: ĐI THẲNG
            q_direct = """
            MATCH (s:BusStop)-[:ON_ROUTE]->(route:BusRoute)<-[:ON_ROUTE]-(e:BusStop)
            WHERE s.id IN $s_ids AND e.id IN $e_ids
            RETURN route.route_no, route.name, s.name AS start_stop, e.name AS end_stop LIMIT 1
            """
            direct = session.run(q_direct, s_ids=s_ids, e_ids=e_ids).single()
            if direct:
                return (f"\n🎯 **TÌM THẤY XE ĐI THẲNG!**\n"
                        f"   🚌 **Đi xe {direct['route.route_no']}**: {direct['route.name']}\n"
                        f"   📍 Đón tại: {direct['start_stop']}\n   🏁 Xuống tại: {direct['end_stop']}")

            # ƯU TIÊN 2: 1 LẦN ĐỔI XE
            q_1_transfer = """
            MATCH (s:BusStop)-[:ON_ROUTE]->(r1:BusRoute)<-[:ON_ROUTE]-(mid:BusStop)
            MATCH (mid)-[:ON_ROUTE]->(r2:BusRoute)<-[:ON_ROUTE]-(e:BusStop)
            WHERE s.id IN $s_ids AND e.id IN $e_ids AND r1 <> r2
            RETURN r1.route_no AS bus1, r2.route_no AS bus2, mid.name AS mid_stop, s.name AS start_stop
            LIMIT 1
            """
            one_stop = session.run(q_1_transfer, s_ids=s_ids, e_ids=e_ids).single()
            if one_stop:
                return (f"\n🔄 **LỘ TRÌNH 1 LẦN ĐỔI XE**\n"
                        f"   1️⃣ Tại '{one_stop['start_stop']}', đón xe **{one_stop['bus1']}**.\n"
                        f"   2️⃣ Đến trạm '**{one_stop['mid_stop']}**' thì xuống.\n"
                        f"   3️⃣ Đón tiếp xe **{one_stop['bus2']}** để đi tiếp.")

            # ƯU TIÊN 3: 2 LẦN ĐỔI XE (Logic mới thêm vào để tránh lỗi không tìm thấy)
            q_2_transfer = """
            MATCH (s:BusStop)-[:ON_ROUTE]->(r1:BusRoute)<-[:ON_ROUTE]-(m1:BusStop)
            MATCH (m1)-[:ON_ROUTE]->(r2:BusRoute)<-[:ON_ROUTE]-(m2:BusStop)
            MATCH (m2)-[:ON_ROUTE]->(r3:BusRoute)<-[:ON_ROUTE]-(e:BusStop)
            WHERE s.id IN $s_ids AND e.id IN $e_ids 
              AND r1 <> r2 AND r2 <> r3
            RETURN r1.route_no, m1.name, r2.route_no, m2.name, r3.route_no LIMIT 1
            """
            two_stops = session.run(q_2_transfer, s_ids=s_ids, e_ids=e_ids).single()
            if two_stops:
                return (f"\n🔀 **LỘ TRÌNH 2 LẦN ĐỔI XE**\n"
                        f"   1. Xe **{two_stops['r1.route_no']}** -> Trạm '{two_stops['m1.name']}'.\n"
                        f"   2. Đổi xe **{two_stops['r2.route_no']}** -> Trạm '{two_stops['m2.name']}'.\n"
                        f"   3. Đổi xe **{two_stops['r3.route_no']}** về đích.")

            return "❌ Quá xa hoặc không có đường đi xe buýt phù hợp (Quá 3 lần đổi xe)."

    def chat(self, text):
        if "đến" in text.lower():
            try:
                parts = text.lower().split("đến")
                start = parts[0].replace("đi từ", "").replace("tìm đường", "").strip()
                end = parts[1].strip()
                if not start or not end: return "⚠️ Vui lòng nhập đủ điểm đi và đến."
                return self.solve_route(start, end)
            except Exception as e:
                return f"⚠️ Lỗi hệ thống: {e}"
        return "🤖 Bot: Hãy nhập 'Đi từ [A] đến [B]'"

if __name__ == "__main__":
    bot = BusBotV13()
    print("\n👋 Tìm đường đi bằng xe bus (Đã Fix Lỗi)!")
    while True:
        q = input("\n👤 Bạn: ")
        if q.lower() in ['exit', 'quit']: break
        print(bot.chat(q))


👋 Tìm đường đi bằng xe bus (Đã Fix Lỗi)!
      🔎 Đang tìm (Normalized): 'đại học tôn đức thắng'...
      🔎 Đang tìm (Normalized): 'trường đại học văn lang'...
      ⚡ Đang quét lộ trình từ 3 trạm đi đến 1 trạm đích...

🎯 **TÌM THẤY XE ĐI THẲNG!**
   🚌 **Đi xe 31**: Bus 31: Đại Học Tôn Đức Thắng - Bến Thành - Đại học Văn Lang
   📍 Đón tại: Đại học Tôn Đức Thắng
   🏁 Xuống tại: Trường Đại học Văn Lang
      🔎 Đang tìm (Normalized): 'trường đại học văn lang'...
      🔎 Đang tìm (Normalized): 'đại học tôn đức thắng'...
      ⚡ Đang quét lộ trình từ 1 trạm đi đến 3 trạm đích...

🎯 **TÌM THẤY XE ĐI THẲNG!**
   🚌 **Đi xe 31**: Bus 31: Đại Học Tôn Đức Thắng - Bến Thành - Đại học Văn Lang
   📍 Đón tại: Trường Đại học Văn Lang
   🏁 Xuống tại: Đại học Tôn Đức Thắng
🤖 Bot: Hãy nhập 'Đi từ [A] đến [B]'
